# 说明
- 推荐使用Google Colab环境，可以直接运行代码；如果使用自建环境，则需要翻墙才能下载模型，需要import torch以及其他省略引用的库。
- 推荐显存16G以上，如有Colab会员，可使用A100训练。
- 样本为1000条，且微调内容比较单一时，推荐epoch5-10，大概训练10分钟。
- 此笔记本参考mlabonne的微调教程。


# 1 导入库和模型

接下来，我们需要导入所需的Python库并加载Llama 3.1 8B模型。

In [ ]:
!pip install -qqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
# 安装unsloth库，它包含了支持Llama模型微调的工具。
!pip install -qqq --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes --progress-bar off
# 安装其他依赖项：xformers（用于高效的内存管理），trl（用于训练和微调），peft（参数高效微调工具），accelerate（加速训练过程），bitsandbytes（支持低比特量化的工具）。

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

# 设置模型的最大序列长度为2048。序列长度决定了模型能够处理的最大输入文本长度。
max_seq_length = 2048

# 使用FastLanguageModel类加载Llama 3.1 8B模型，启用4bit量化以减少内存占用。
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,  # 如果硬件支持bfloat16精度，可以指定dtype为torch.bfloat16以进一步减少内存占用。
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. 为PEFT（参数高效微调）准备模型

我们将模型设置为使用PEFT技术，并启用LoRA（低秩适应）进行参数微调，以减少训练所需的计算资源。

In [ ]:
# 将模型配置为使用PEFT技术，特别是通过LoRA进行微调。
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA的秩值，控制额外的参数数量。较高的值增加模型容量，但也增加内存需求。
    lora_alpha=16,  # LoRA的alpha参数，控制学习率的缩放。
    lora_dropout=0,  # LoRA中的dropout率，设置为0表示不使用dropout。
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    # 指定哪些模块将使用LoRA进行微调。包括自注意力和MLP层的投影矩阵。
    use_rslora=True,  # 使用RSLora技术，进一步优化内存和计算效率。
    use_gradient_checkpointing="unsloth"  # 启用梯度检查点，以降低内存消耗。
)

# 输出模型中可训练的参数数量，以验证PEFT设置是否成功。
print(model.print_trainable_parameters())

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196
None


## 3. 准备数据集和分词器

接下来，我们加载训练数据集，并准备分词器。分词器将文本数据转换为模型可以处理的输入格式。

In [ ]:
# # 获取一个聊天模板，用于格式化输入数据，使其符合模型的输入要求。
# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template="chatml",  # 使用"chatml"模板，该模板适用于聊天对话格式的输入。
#     mapping={"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}
#     # 映射数据集中不同字段，以符合模板要求。将"user"映射为"human"，将"assistant"映射为"gpt"。
# )

# # 定义一个函数，将数据集中的对话格式化为模型输入所需的文本格式。
# def apply_template(examples):
#     messages = examples["conversations"]  # 提取对话内容。
#     text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
#     # 使用分词器的聊天模板将对话格式化为模型输入文本。
#     return {"text": text}  # 返回格式化后的文本。

# # 加载数据集，选择训练集部分。
# dataset = load_dataset("mlabonne/FineTome-100k", split="train")
# # 将自定义格式化函数应用于整个数据集，以便于模型输入。
# dataset = dataset.map(apply_template, batched=True)

import pandas as pd
from datasets import load_dataset, Dataset  # 确保导入了 Dataset

# 获取一个聊天模板，用于格式化输入数据，使其符合模型的输入要求。
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # 使用"chatml"模板，该模板适用于聊天对话格式的输入。
    mapping={"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}
    # 映射数据集中不同字段，以符合模板要求。将"user"映射为"human"，将"assistant"映射为"gpt"。
)

# 定义一个函数，将数据集中的对话格式化为模型输入所需的文本格式。
def apply_template(examples):
    messages = examples["conversations"]  # 提取对话内容。
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    # 使用分词器的聊天模板将对话格式化为模型输入文本。
    return {"text": text}  # 返回格式化后的文本。

# 加载本地数据集
df = pd.read_parquet('/content/output.parquet')
dataset = Dataset.from_pandas(df)

# 将自定义格式化函数应用于整个数据集，以便于模型输入。
dataset = dataset.map(apply_template, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/876 [00:00<?, ? examples/s]

## 4. 模型训练

现在，我们配置训练参数并开始训练模型。

In [ ]:
# 使用SFTTrainer（顺序微调训练器）配置和训练模型。
trainer = SFTTrainer(
    model=model,  # 使用之前准备好的模型。
    tokenizer=tokenizer,  # 使用准备好的分词器。
    train_dataset=dataset,  # 训练数据集。
    dataset_text_field="text",  # 数据集中包含文本的字段名称。
    max_seq_length=max_seq_length,  # 模型最大序列长度。
    dataset_num_proc=2,  # 用于处理数据集的并行进程数量，增加此值可以加快预处理速度。
    packing=True,  # 启用数据打包技术，减少训练时的内存占用。
    args=TrainingArguments(
        learning_rate=3e-4,  # 设置学习率，控制模型参数更新的速度。
        lr_scheduler_type="linear",  # 使用线性学习率调度器，随着训练过程逐渐降低学习率。
        per_device_train_batch_size=4,  # 每个GPU设备的训练批次大小，较小的值减少显存占用。
        gradient_accumulation_steps=4,  # 梯度累积步数，有助于在小批次情况下模拟大批次训练。
        num_train_epochs=10,  # 训练周期数，设置为表示只训练一次整个数据集。
        fp16=not is_bfloat16_supported(),  # 如果硬件支持bfloat16，则使用它，否则使用fp16。
        bf16=is_bfloat16_supported(),  # 同上，启用bfloat16支持。
        logging_steps=1,  # 每一步都记录日志，方便监控训练过程。
        optim="adamw_8bit",  # 使用AdamW优化器，8bit版本可减少内存使用。
        weight_decay=0.01,  # 权重衰减系数，防止过拟合。
        warmup_steps=10,  # 热身步数，在开始的几个步骤中逐渐增加学习率。
        output_dir="output",  # 输出目录，用于保存模型和日志。
        seed=0,  # 随机种子，确保实验的可重复性。
    ),
)

# 开始训练模型。
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 32 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.344800
2,1.384200
3,1.237200
4,0.892700
5,0.830100
6,0.631700
7,0.634100
8,0.511300
9,0.436300
10,0.429200


TrainOutput(global_step=20, training_loss=0.5386555336415768, metrics={'train_runtime': 197.5545, 'train_samples_per_second': 1.62, 'train_steps_per_second': 0.101, 'total_flos': 2.967549134241792e+16, 'train_loss': 0.5386555336415768, 'epoch': 10.0})

## 5. 推理

训练完成后，我们可以加载微调后的模型进行推理。

In [ ]:
# 为推理重新加载模型，将其转换为推理模式。
model = FastLanguageModel.for_inference(model)

# 定义要输入模型的对话消息。
messages = [
    {"from": "human", "value": "你是谁？"},
]

# 使用分词器将消息格式化为模型可处理的输入，并将其转换为张量形式。
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")  # 将输入数据加载到GPU（cuda）上。

# 定义文本流输出器，实时显示生成的文本。
text_streamer = TextStreamer(tokenizer)

# 使用模型生成文本，设置最大生成的token数为128，并启用缓存以加快生成速度。
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=2048, use_cache=True)

<|im_start|>user
用C++写一段Helloworld代码,补全详细的中文注释，确保其准确性<|im_end|>
<|im_start|>assistant
using namespace std;

int main() {
    // 1. 声明变量
    string name = "World";

    // 2. 打印Hello World
    cout << "Hello " << name << endl;

    return 0;
}
<|im_end|>


## 6. 保存微调后的模型

最后，我们将微调后的模型保存。

In [ ]:
# # 将微调后的模型保存为合并的16bit精度格式，适用于推理。
# model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")

# # 将模型上传到Hugging Face模型中心，方便共享和部署。
# model.push_to_hub_merged("mlabonne/FineLlama-3.1-8B", tokenizer, save

# _method="merged_16bit")

# # 还可以将模型保存为GGUF格式，这是一种支持更高效推理的量化格式。
# model.save_pretrained_gguf("model", tokenizer, "q8_0")

# # 定义一组量化方法，用于以不同的量化精度保存和上传模型。
# quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]

# # 使用不同的量化方法将模型上传到模型中心，方便不同需求的用户使用。
# for quant in quant_methods:
#     model.push_to_hub_gguf("mlabonne/FineLlama-3.1-8B-GGUF", tokenizer, quant)

# 保存合并后的16位精度模型到本地目录
model.save_pretrained_merged("./model", tokenizer, save_method="merged_16bit")

# 保存量化后的模型到本地目录（使用不同的量化方法）
model.save_pretrained_gguf("./model", tokenizer, "q8_0")

# # 定义多种量化方法，分别保存模型
# quant_methods = ["q8_0"]
# for quant in quant_methods:
#     model.save_pretrained_gguf(f"./model_{quant}", tokenizer, quant)

print("所有模型已成功保存到本地目录。")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 54.04 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 68.68it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 61.24 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 64.56it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at ./model into q8_0 GGUF format.
The output location will be ././model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,      